In [1]:
%matplotlib inline
import utils#; reload(utils)
from utils import *
from __future__ import division, print_function


import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras.utils.data_utils import get_file
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
#path = "data/fish/sample/"
path = "data/nof_excl/"
batch_size=32
target_size = (360, 640)

In [3]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 2912 images belonging to 7 classes.
Found 400 images belonging to 7 classes.
Found 12153 images belonging to 1 classes.


In [4]:
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

In [5]:
# conv_val_feat = load_array(path+'results/conv_val_640.dat')
# conv_trn_feat = load_array(path+'results/conv_trn_640.dat')
# conv_test_feat = load_array(path+'results/conv_test_640.dat')

Create a model that represents common parts of VGG 


In [6]:
vgg640 = Vgg16BN((360, 640)).model
vgg640.pop()
print (vgg640.input_shape, vgg640.output_shape)
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


(None, 3, 360, 640) (None, 512, 22, 40)


A few convnet layers on top of vgg conv layers

In [6]:
nf=128; p=0.

from keras.layers.pooling import GlobalMaxPooling2D
def get_additional_layers(conv_layers):
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(7,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
#         GlobalMaxPooling2D(),
        Activation('softmax')
    ]



In [7]:
conv_layers,_ = split_at(vgg640, Convolution2D)
model = Sequential(get_additional_layers(conv_layers))
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


th


### Training Augmentation

In [5]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, zoom_range=0.05,
                               height_shift_range=0.05, shear_range=0.05,
                                      horizontal_flip=True)

In [19]:
train_flow = gen.flow_from_directory(path+"train", target_size=(360, 640),
            class_mode=None, shuffle=False, batch_size=1)
train_batches = np.concatenate([train_flow.next() for i in range(train_flow.nb_sample)])



Found 2912 images belonging to 7 classes.


In [22]:
conv_trn_feat = vgg640.predict(train_batches, batch_size=32, verbose=1)

2912/2912 [==============================] - 361s   


In [23]:
# model.fit_generator(train_batches, samples_per_epoch=train_batches.nb_sample, nb_epoch=1,
#             validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

# vgg640

model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=5, 
             validation_data=(conv_val_feat, val_labels))

Train on 2912 samples, validate on 400 samples
Epoch 1/5
2912/2912 [==============================] - 14s - loss: 0.7993 - acc: 0.7284 - val_loss: 1.2626 - val_acc: 0.6800
Epoch 2/5
2912/2912 [==============================] - 14s - loss: 0.1786 - acc: 0.9481 - val_loss: 1.0820 - val_acc: 0.7400
Epoch 3/5
2912/2912 [==============================] - 14s - loss: 0.0521 - acc: 0.9870 - val_loss: 0.3246 - val_acc: 0.9175
Epoch 4/5
2912/2912 [==============================] - 14s - loss: 0.0196 - acc: 0.9962 - val_loss: 0.3571 - val_acc: 0.9175
Epoch 5/5
2912/2912 [==============================] - 14s - loss: 0.0050 - acc: 1.0000 - val_loss: 0.3277 - val_acc: 0.9175


In [27]:
# model.optimizer.lr = 1e-4

# for i in range(3):
#     #calculate conv features for the new batch
#     train_flow = gen.flow_from_directory(path+"train", target_size=(360, 640),
#             class_mode=None, shuffle=False, batch_size=1)
#     train_batches = np.concatenate([train_flow.next() for i in range(train_flow.nb_sample)])
#     conv_trn_feat = vgg640.predict(train_batches, batch_size=32, verbose=1)
    
#     #fit model with small learning rate on new batch
#     model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=10, 
#              validation_data=(conv_val_feat, val_labels))

### An end-to-end convnet without pre-computations

In [11]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, zoom_range=0.05,
                                      channel_shift_range=10, height_shift_range=0.1, shear_range=0.05,
                                      fill_mode='nearest',horizontal_flip=True)

In [7]:
train_flow = gen.flow_from_directory(path+"train", target_size=target_size,
            class_mode="categorical", shuffle=True, batch_size=batch_size)

val_flow = image.ImageDataGenerator().flow_from_directory(path+"valid", target_size=target_size, batch_size=batch_size,
                                                   class_mode='categorical', shuffle=True)

Found 2912 images belonging to 7 classes.
Found 400 images belonging to 7 classes.


In [6]:
nf=128; p=0.

from keras.layers.pooling import GlobalMaxPooling2D
def get_additional_layers(conv_layers):
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(7,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
#         GlobalMaxPooling2D(),
        Activation('softmax')
    ]



In [7]:
model = Vgg16BN((360, 640)).model
model.pop()

for layer in model.layers: #Freeze feature layers
    layer.trainable = False
    
    
    
conv_layers,_ = split_at(model, Convolution2D)

for l in get_additional_layers(conv_layers):
    model.add(l)
    
print (model.input_shape, model.output_shape)
model.compile(Adam(lr=0.001), 'categorical_crossentropy', metrics=['accuracy'])


th
(None, 3, 360, 640) (None, 7)


In [8]:
len(get_additional_layers(conv_layers))

th


14

In [26]:

model.fit_generator(train_flow, samples_per_epoch=train_flow.nb_sample, nb_epoch=5,
                                 validation_data=val_flow, nb_val_samples=val_flow.nb_sample)

Epoch 1/5
2912/2912 [==============================] - 436s - loss: 0.0712 - acc: 0.9763 - val_loss: 0.2529 - val_acc: 0.9150
Epoch 2/5
2912/2912 [==============================] - 436s - loss: 0.0822 - acc: 0.9742 - val_loss: 0.2888 - val_acc: 0.9400
Epoch 3/5
2912/2912 [==============================] - 436s - loss: 0.0763 - acc: 0.9739 - val_loss: 0.2587 - val_acc: 0.9525
Epoch 4/5
2912/2912 [==============================] - 436s - loss: 0.0817 - acc: 0.9718 - val_loss: 0.1914 - val_acc: 0.9575
Epoch 5/5
2912/2912 [==============================] - 436s - loss: 0.0578 - acc: 0.9818 - val_loss: 0.2141 - val_acc: 0.9600


In [10]:
# model.save_weights(path+"results/with_aug_after_10epochs_val_acc_0.96.hd5")
# model.load_weights(path+"results/with_aug_after_10epochs_val_acc_0.96.hd5")

In [12]:
model.optimizer.lr = 1e-4
model.fit_generator(train_flow, samples_per_epoch=train_flow.nb_sample, nb_epoch=5,
                                 validation_data=val_flow, nb_val_samples=val_flow.nb_sample)

Epoch 1/5
2912/2912 [==============================] - 438s - loss: 0.0474 - acc: 0.9839 - val_loss: 0.1800 - val_acc: 0.9625
Epoch 2/5
2912/2912 [==============================] - 439s - loss: 0.0302 - acc: 0.9907 - val_loss: 0.1646 - val_acc: 0.9650
Epoch 3/5
2912/2912 [==============================] - 439s - loss: 0.0240 - acc: 0.9928 - val_loss: 0.1805 - val_acc: 0.9675
Epoch 4/5
2912/2912 [==============================] - 439s - loss: 0.0193 - acc: 0.9938 - val_loss: 0.1725 - val_acc: 0.9675
Epoch 5/5
2912/2912 [==============================] - 439s - loss: 0.0157 - acc: 0.9959 - val_loss: 0.1827 - val_acc: 0.9675


In [9]:
# model.save_weights(path+"models/with_aug_after_5epochs_e_4_val_acc_9675.hd5")
model.load_weights(path+"models/with_aug_after_5epochs_e_4_val_acc_9675.hd5")

#### Make the original top 2 layers trainable
After getting good enough accuracy on default VGG weights, it's time to allow the two top layers of original VGG to adapt

In [15]:
# model.layers
#model.layers.trainable = true 

### Test Augmentation

In [21]:
test_datagen = gen
num_iterations = 5
preds = np.zeros((len(test_filenames), 7)) #7 fish types 

for i in range(num_iterations):
    print ("Starting prediction round {}".format(i+1))
    
    test_gen = test_datagen.flow_from_directory(path+"test", target_size=target_size, batch_size=32,
                                                        class_mode=None, shuffle=False)
    preds += model.predict_generator(test_gen, val_samples=len(test_filenames))
    
    

Starting prediction round 1
Found 12153 images belonging to 1 classes.
Starting prediction round 2
Found 12153 images belonging to 1 classes.
Starting prediction round 3
Found 12153 images belonging to 1 classes.
Starting prediction round 4
Found 12153 images belonging to 1 classes.
Starting prediction round 5
Found 12153 images belonging to 1 classes.


In [22]:
preds[0:1]

array([[  4.1142e+00,   5.5797e-04,   1.5526e-03,   5.0141e-04,   1.2738e-03,   1.2471e-05,
          8.8194e-01]])

In [24]:
averaged_preds = preds / num_iterations

In [25]:
averaged_preds.shape

(12153, 7)

In [26]:
# save_array(path+"results/preds_with_train_n_test_augmentation_val_acc_0.9675.dat",averaged_preds)
save_array(path+"results/stg2_preds_with_train_n_test_augmentation_val_acc_0.9675.dat",averaged_preds)

In [9]:
averaged_preds = load_array(path+"results/stg2_preds_with_train_n_test_augmentation_val_acc_0.9675.dat")

## Dense Layer on Top

This will serve as the second model for bagging.
Very light augmentation for this model 

In [8]:
from vgg16bn import Vgg16BN

vgg640 = Vgg16BN(size=(360, 640)).model

vgg640.pop()
print (vgg640.input_shape, vgg640.output_shape)
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])


conv_layers,fc_layers = split_at(vgg640, Convolution2D)
conv_model = Sequential(conv_layers)




(None, 3, 360, 640) (None, 512, 22, 40)


In [8]:
# trn = load_array(path+'results/trn_640.dat')
# val = load_array(path+'results/val_640.dat')
# # test = load_array(path+'results/val_640.dat')

# conv_feat = conv_model.predict(trn)
# conv_val_feat = conv_model.predict(val)

In [48]:
# conv_val_feat = load_array(path+'results/conv_val_640.dat')
# conv_trn_feat = load_array(path+'results/conv_trn_640.dat')
# conv_test_feat = load_array(path+'results/conv_test_640.dat')

In [9]:
train_datagen = image.ImageDataGenerator(
        shear_range=0.05,
        zoom_range=0.1,
        rotation_range=7.,
        width_shift_range=0.07,
        height_shift_range=0.07,
    channel_shift_range=5,
        horizontal_flip=True)


# set shuffle=False if pre-

# aug_trn_batches = train_datagen.flow_from_directory(path+'train', target_size=target_size,
#            class_mode='categorical', shuffle=False, batch_size=batch_size)


# aug_val_batches = image.ImageDataGenerator().flow_from_directory(path+"valid", target_size=target_size, batch_size=batch_size,
#                                                    class_mode='categorical', shuffle=False)

In [8]:
# da_conv_featx2 = vgg640.predict_generator(aug_trn_batches, aug_trn_batches.nb_sample*2)
# save_array(path+"results/da_conv_featx2.dat",da_conv_featx2)
da_conv_featx2 = load_array(path+"results/da_conv_featx2.dat")

In [21]:
# conv_trn_feat = load_array(path+'results/conv_trn_640.dat')
# conv_val_feat = load_array(path+'results/conv_val_640.dat')

In [9]:
#Let's include the real training data as well in its non-augmented form.
da_conv_feat = np.concatenate([da_conv_featx2, conv_trn_feat])


In [13]:

#Since we've now got a dataset 6x bigger than before, we'll need to copy our labels 6 times too.
da_trn_labels = np.concatenate([trn_labels]*3)



In [ ]:

# save_array(path+'results/conv_val_feat_640.dat', conv_val_feat)
# save_array(path+'results/conv_feat_640.dat', conv_feat)

In [10]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/3),
        Flatten(),
        Dense(2048, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(2048, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(7, activation='softmax')
    ]

In [11]:
p = 0.6
# bn_model = vgg_ft_bn(1)
# conv_layers,fc_layers = split_at(vgg640, Convolution2D)

dense_model = Sequential(get_bn_layers(p))
dense_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])



In [23]:

dense_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=5, 
             validation_data=(conv_val_feat, val_labels))

Train on 8736 samples, validate on 400 samples
Epoch 1/5
8736/8736 [==============================] - 63s - loss: 1.1111 - acc: 0.7491 - val_loss: 0.1354 - val_acc: 0.9750
Epoch 2/5
8736/8736 [==============================] - 63s - loss: 0.2143 - acc: 0.9439 - val_loss: 0.1067 - val_acc: 0.9725
Epoch 3/5
8736/8736 [==============================] - 63s - loss: 0.1318 - acc: 0.9669 - val_loss: 0.1335 - val_acc: 0.9750
Epoch 4/5
8736/8736 [==============================] - 63s - loss: 0.0821 - acc: 0.9783 - val_loss: 0.1334 - val_acc: 0.9750
Epoch 5/5
8736/8736 [==============================] - 63s - loss: 0.0655 - acc: 0.9844 - val_loss: 0.1206 - val_acc: 0.9750


In [17]:
# dense_model.save_weights(path+"models/dense_augx2_1e3_5epoc_valacc_0.9750.h5")
dense_model.load_weights(path+"models/dense_augx2_1e3_5epoc_valacc_0.9750.h5")

In [18]:
dense_model.optimizer.lr = 1e-6

dense_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels))

Train on 8736 samples, validate on 400 samples
Epoch 1/2
8736/8736 [==============================] - 63s - loss: 0.0352 - acc: 0.9910 - val_loss: 0.1238 - val_acc: 0.9750
Epoch 2/2
8736/8736 [==============================] - 63s - loss: 0.0384 - acc: 0.9894 - val_loss: 0.1188 - val_acc: 0.9800


In [13]:
# dense_model.save_weights(path+"models/dense_augx2_1e6_2epoc_valacc_0.9800.h5")
dense_model.load_weights(path+"models/dense_augx2_1e6_2epoc_valacc_0.9800.h5")



In [20]:
#cleanup
# del da_conv_feat, conv_val_feat, da_conv_featx2, conv_trn_feat

### Making Predictions 

In [15]:
predictions = np.zeros(shape=(len(test_filenames), 7))

nb_aug = 5
for aug in range(nb_aug):
    print ("Starting prediction round ", aug+1)

    test_batches = train_datagen.flow_from_directory(path+"test", target_size=target_size,
                class_mode=None, shuffle=False, batch_size=32)


    conv_test_feat = vgg640.predict_generator(test_batches, len(test_filenames))
    predictions += dense_model.predict(conv_test_feat, batch_size=32)
    
    del conv_test_feat # release memory to avoid memory exception
    


Starting prediction round  1
Found 12153 images belonging to 1 classes.


INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock


Starting prediction round  2
Found 12153 images belonging to 1 classes.
Starting prediction round  3
Found 12153 images belonging to 1 classes.
Starting prediction round  4
Found 12153 images belonging to 1 classes.
Starting prediction round  5
Found 12153 images belonging to 1 classes.


In [16]:
predictions[0:1]

array([[  4.7667e+00,   1.6435e-02,   1.1405e-02,   1.0580e-03,   3.6808e-02,   6.1723e-04,
          1.6694e-01]])

In [17]:
predictions /= nb_aug

output from full convnet for comparison: 

array([[  4.1142e+00,   5.5797e-04,   1.5526e-03,   5.0141e-04,   1.2738e-03,   1.2471e-05,
          8.8194e-01]])

In [18]:
save_array(path+"results/stg2_preds_dense_augx2_1e6_valacc_0.9800.dat",predictions)

In [5]:
predictions = load_array(path+"results/stg2_preds_dense_augx2_1e6_valacc_0.9800.dat")

In [7]:
predictions[0]*5

array([  4.7667e+00,   1.6435e-02,   1.1405e-02,   1.0580e-03,   3.6808e-02,   6.1723e-04,
         1.6694e-01])

### Pseudo labeling

In [69]:
class MixIterator(object):
    def __init__(self, iters):
        self.iters = iters
        self.multi = type(iters) is list
#         if self.multi:
#             self.N = sum([it[0].N for it in self.iters])
#         else:
        self.N = sum([it.N for it in self.iters])

    def reset(self):
        for it in self.iters: it.reset()

    def __iter__(self):
        return self

    def next(self, *args, **kwargs):
#         if self.multi:
#             nexts = [[next(it) for it in o] for o in self.iters]
#             n0s = np.concatenate([n[0] for n in o])
#             n1s = np.concatenate([n[1] for n in o])
#             return (n0, n1)
#         else:
        nexts = [next(it) for it in self.iters]
        n0 = np.concatenate([n[0] for n in nexts])
        n1 = np.concatenate([n[1] for n in nexts])
        return (n0, n1)


In [70]:
val_pseudo = dense_model.predict(conv_val_feat, batch_size=batch_size)
test_pseudo = dense_model.predict(conv_test_feat, batch_size=batch_size)

In [71]:
gen = image.ImageDataGenerator()


trn_batch  = gen.flow(da_conv_feat, da_trn_labels,batch_size=44)
val_batch  = gen.flow(conv_val_feat,val_pseudo,batch_size=4)
test_batch = gen.flow(conv_test_feat,test_pseudo,batch_size=16)


In [72]:
mi = MixIterator([trn_batch, val_batch,test_batch])



In [73]:
dense_model.optimizer.lr = 1e-3
dense_model.fit_generator(mi, mi.N, nb_epoch=8, validation_data=(conv_val_feat, val_labels))


Epoch 1/8
7224/7224 [==============================] - 230s - loss: 0.1263 - acc: 0.9695 - val_loss: 0.0857 - val_acc: 0.9775
Epoch 2/8
7252/7224 [==============================] - 230s - loss: 0.1206 - acc: 0.9732 - val_loss: 0.0872 - val_acc: 0.9775
Epoch 3/8
7252/7224 [==============================] - 230s - loss: 0.1197 - acc: 0.9750 - val_loss: 0.0898 - val_acc: 0.9775
Epoch 4/8
 256/7224 [>.............................] - ETA: 206s - loss: 0.1249 - acc: 0.9648

KeyboardInterrupt: 

In [74]:
dense_model.optimizer.lr = 1e-5
dense_model.fit_generator(mi, mi.N, nb_epoch=5, validation_data=(conv_val_feat, val_labels), callbacks=[early_stopping])

Epoch 1/5
7252/7224 [==============================] - 235s - loss: 0.1130 - acc: 0.9756 - val_loss: 0.0925 - val_acc: 0.9750
Epoch 2/5
7252/7224 [==============================] - 231s - loss: 0.1070 - acc: 0.9793 - val_loss: 0.0933 - val_acc: 0.9750
Epoch 3/5
7252/7224 [==============================] - 231s - loss: 0.1098 - acc: 0.9790 - val_loss: 0.0949 - val_acc: 0.9725


In [75]:
dense_model.save_weights(path+"models/dense_psudeo_augx1_1e4_13epoc_valacc_0.9775.h5")

### Submission

In [22]:
#old code, paths are inconsistent !ignore

import pandas as pd

classes = ['ALB', 'BET', 'DOL', 'LAG','NoF', 'OTHER', 'SHARK', 'YFT']
# nofish_prob = load_array("data/fishnofish/results/preds_640_loss_0.0244_acc_0.9913_val_loss_0.2086_val_acc_0.9778.dat")
# nofish_prob = load_array("data/fishnofish/results/pred_stg1_fullconv_aug_no_pseudo.dat")
# nofish_prob = load_array("data/fishnofish/results/final_stg1_weights_ensumble.dat")

nofish_prob = load_array("data/fishnofish/results/final_stg1_weights_ensumble_with_pseudo.dat")



nofish_prob = nofish_prob[:, 1] #nofish class


In [ ]:
# stg1
preds = load_array(path+"results/preds_light_aug_cropped_v1.dat")
# preds_denseaug = load_array(path+"results/preds_dense_precomputed_aug_1e7_val_acc_0.9625.dat")

#stg 2
preds_denseaug = load_array(path+"results/stg2_preds_dense_precomputed_aug_1e7_val_acc_0.9625.dat")
"results/stg2_preds_dense_augx2_1e6_valacc_0.9800.dat"

subm_name = path+'submit/preds_cropped_v1_denseaugprecompute.gz'


In [ ]:
preds+= preds_denseaug
preds /= 2.0

In [ ]:
def weight_predictions(predictions):
    """Weights predictions based on probability image contains a fish as predicted by fish detector model"""
    no_fish = predictions[:, 4]
    fish = np.delete(predictions, 4, axis=1)

    weights = -1. * (no_fish - 1.)
    weights = weights.reshape(1000, 1)

    fish = weights * fish
    preds = np.insert(fish, 4, no_fish, axis=1)

    return preds


In [ ]:
with_nofish_probs = np.insert(preds, 4, nofish_prob, axis=1)
weighted_preds = weight_predictions(with_nofish_probs)

In [ ]:
f = 0.85#0.82
subm = np.clip(weighted_preds, (1-f)/7, f)